# RAG

In [20]:
import dotenv

from agents import Agent, Runner, trace, function_tool

import chromadb

dotenv.load_dotenv()

True

Create a static calorie table that we can use as a tool:

In [21]:
# We populated the RAG with the data from the data/calories.csv file in the rag_setup.ipynb notebook

chroma_client = chromadb.PersistentClient(path="../chroma")
nutrition_db = chroma_client.get_collection(name="nutrition_db")

In [23]:
results = nutrition_db.query(query_texts=["banana"], n_results=2)
for i, doc in enumerate(results["documents"][0]):
    print(sorted(results["metadatas"][0][i].items()))
    print(doc)
    print("\n")

[('calories_per_100g', 89.0), ('food_category', 'fruits'), ('food_item', 'banana'), ('keywords', 'banana_fruits'), ('kj_per_100g', 374.0), ('serving_info', '100g')]
Food: Banana
        Category: Fruits
        Nutritional Information:
        - Calories: 89 per 100g
        - Energy: 374 kJ per 100g
        - Serving size reference: 100g

        This is a fruits food item that provides 89 calories per 100 grams.


[('calories_per_100g', 50.0), ('food_category', '(fruit)juices'), ('food_item', 'banana juice'), ('keywords', 'banana_juice_(fruit)juices'), ('kj_per_100g', 210.0), ('serving_info', '100ml')]
Food: Banana Juice
        Category: (Fruit)Juices
        Nutritional Information:
        - Calories: 50 per 100g
        - Energy: 210 kJ per 100g
        - Serving size reference: 100ml

        This is a (fruit)juices food item that provides 50 calories per 100 grams.




In [24]:
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function too look up calorie information.

    Args:
        query: The food item to look up.
        collection: The Chroma collection to use.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

In [25]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information.
    You give concise answers.
    If you need to look up calorie information, use the calorie_lookup_tool.
    """,
    tools=[calorie_lookup_tool],
)

In [27]:
with trace("Nutrition Assistant with RAG"):
    result = await Runner.run(
        calorie_agent,
        "How many calories are in total in a banana and an apple? Also give calories per 100g",
    )
    print(result.final_output)

- Total (one medium banana + one medium apple): about 200 kcal.
- Per 100 g: Banana 89 kcal; Apple 52 kcal.
